# Libraries importation

In [1]:
# Import all libraries needed 

import matplotlib.pyplot as plt
import pandas as pd
import sys 
import scipy
import numpy as np

from scipy.signal import *
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import metrics

In [2]:
sys.path.insert(0, '../packaging_PIR')
from neural_data_treatment_pkg.PrintFunctions import *
from neural_data_treatment_pkg.AdaBandFlt import *

In [3]:
# Enable inline plotting
%matplotlib inline
# Enable outline plotting
%matplotlib tk

# Data Loading (execute only the one corresponding to your device)

In [69]:
# file path of csv file

Location = r'/Users/sylva/Documents/SUPAERO/2A/PIR/Data/data_no_burst/539W6cbaseRaw.txt'

In [5]:
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

In [ ]:
# complete this one with your own path

#Location = 

In [11]:
# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

C:\Users\sylva\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
df.columns

Index(['El 41       ', 'El 42       ', 'El 44       ', 'El 54       ',
       'El 55       ', 'El 56       ', 'El 58       ', 'El 86       ',
       'El 15       ', 'El 51       '],
      dtype='object')

# Defining the sampling frequency and the alignement method for the rest of the notebook

In [13]:
fs = 50000
align_method = 'indice_1er_depass'

# Cuting and filtering the signal

In [46]:
#####################################################################################################################
####  BANK OF PARTS OF DATA
size = -1
all_raw_data = df #Entire recording from all electrodes

#full_signal = df.iloc[:,1] #Entire recording from electrode 58
full_signal = df.loc[:size,'El 58       '] #Entire recording from electrode 58

electrode_ref = df.loc[:size,'El 15       ']

# Desired cutoff frequencies (in Hz).
lowcut = 300.0
highcut = 5000.0

#y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)
y = butter_bandpass_filter(df.iloc[:size,6], lowcut, highcut, fs, order=5)
y_ref = butter_bandpass_filter(df.iloc[:size,8],lowcut,highcut,fs,order=5)

filtereddf = pd.DataFrame(y)
filtereddf.index = df.index[:size]

filtereddf_ref = pd.DataFrame(y_ref)
filtereddf_ref.index = df.index[:size]


signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode
signal_filtered_ref = filtereddf_ref.iloc[:,0]


###########################
## Signal de 20s

xminspike = int(np.round(12548*(fs/1000)))
xmaxspike = int(np.round(13000*(fs/1000)))

burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

C:\Users\sylva\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


## selecting the signal or the part of the signal

In [47]:
signal = signal_filtered

# Run the noise initialisation

In [48]:
noise_levels = init_noise_levels(signal_filtered, fs, 
                                  noise_window_size = 0.01,
                                  required_valid_windows = 20,
                                  old_noise_level_propagation = 0.8, 
                                  test_level = 5,
                                  estimator_type = "RMS",
                                  percentile_value = 25,
                                  plot_estimator_graph = True)

# Find the spikes

In [49]:
spike_info = find_spikes(signal, noise_levels, fs,
                           window_size = 0.002,
                           noise_window_size = 0.01,
                           threshold_factor = 4,
                           positive_threshold_factor = 0.33,
                           maxseparation = 0.001,
                           time_checkmaxlocal = 0.0002)

# Record the spikes

In [50]:
spike_data = record_spikes(signal, fs, spike_info,
                              align_method,
                              t_before = 0.001,
                              t_after = 0.002)

In [51]:
print_spikes(spike_data,
             t_before_alignement = 0.001,
             first_spike = 0,
             last_spike = -1,
             fs = fs,
             randomize = True,
             nb_spike = 20,
             y_lim_min = -50,
             y_lim_max = 60)

In [52]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                                              align_method,
                                              t_before = 0.001,
                                              t_after = 0.002)

In [53]:
print_spikes_oneline(signal, spike_data_oneline)

# PCA and Clustering

In [54]:
pca_data = np.array(spike_data.values).transpose()

In [62]:
fig = plt.figure(4, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()
pca = PCA(n_components=3)
pca.fit(pca_data)
X = pca.transform(pca_data)

# Reorder the labels to have colors matching the cluster results
#y = np.choose(y, [1, 0]).astype(np.float)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

plt.show()

In [63]:
from sklearn.cluster import KMeans

In [64]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
labels = kmeans.labels_

In [65]:
print_clusters_3d(labels, X)

# Print the spikes clusterized

In [66]:
print_spikes_clusterized(spike_data,
                             labels,
                             t_before_alignement = 0.001,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = fs)

In [67]:
spike_data_clusterized_oneline = record_spikes_clusterized_oneline(signal, 
                                                                      fs, 
                                                                      spike_info,
                                                                      align_method,
                                                                       labels,
                                                                      t_before = 0.001,
                                                                      t_after = 0.002)

The labels located in labels will be used


In [68]:
print_spikes_clusterized_oneline(signal, spike_data_clusterized_oneline,
                                     y_lim_min = -20,
                                     y_lim_max = 20)